In [1]:
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt
import numpy as np
from sqlite3 import connect
from markdown import MarkDownSeries, MarkDownTable

### Reading in our data

In [2]:
RawData = pd.read_csv('vgchartz-2024.csv')

### Looking at our code from the last chapter. 

The code processes the RawData DataFrame by:

- Converting the release_date column to `datetime`

- Removing specified **columns**: 'img' and 'last_update'

- Drops rows with missing 'release_date' 

- Fills missing values with **zeros** in certain columns

- Converts the 'console' column values to lowercase

```python
RawData['release_date'] = pd.to_datetime(RawData['release_date'])
RawData.drop(['img', 'last_update'], axis=1, inplace=True)
RawData = RawData.dropna(subset=['release_date'])
columns_to_fill_zero = ['critic_score', 'total_sales', 'na_sales', 'jp_sales', 'pal_sales', 'other_sales']
RawData.loc[:, columns_to_fill_zero] = RawData.loc[:, columns_to_fill_zero].fillna(0)
RawData['console'] = RawData['console'].str.lower()
```

In [3]:
RawData['release_date'] = pd.to_datetime(RawData['release_date'])
RawData.drop(['img', 'last_update'], axis=1, inplace=True)
RawData = RawData.dropna(subset=['release_date'])
columns_to_fill_zero = ['critic_score', 'total_sales', 'na_sales', 'jp_sales', 'pal_sales', 'other_sales']
RawData.loc[:, columns_to_fill_zero] = RawData.loc[:, columns_to_fill_zero].fillna(0)
RawData['console'] = RawData['console'].str.lower()

In [5]:
RawData['console'].value_counts()

console
pc      10477
ps2      3511
ds       3166
ps       2694
ps4      2102
        ...  
fmt         3
cd32        2
aco         1
bbcm        1
c128        1
Name: count, Length: 79, dtype: int64

Problem: We have consoles for the sales data, but we don't have who the manufacturer is for the consoles. To solve this we will map the consoles to a manfacture using a dictionary. 

Mapping console to make new manufacture column
- Make dictionary for mapping 
- Flatten into a single list 
- Checking for missing items 
- Create conditions and values for np.select
- Assign console manufacturers


In [6]:
categories = {
    'nintendo': ['3ds', 'dsiw', 'dsi', 'ds', 'wii', 'wiiu', 'ns', 'gb', 'gba', 'nes', 'snes', 'gbc', 'n64', 'vb', 'gc', 'vc','ww'],
    'pc': ['linux', 'osx', 'pc', 'arc', 'all', 'fmt', 'c128', 'aco'],
    'xbox': ['x360', 'xone', 'series', 'xbl', 'xb', 'xs'],
    'sony': ['ps', 'ps2', 'ps3', 'ps4', 'ps5', 'psp', 'psv', 'psn', 'cdi'],
    'mobile': ['ios', 'and', 'winp', 'ngage', 'mob'],
    'sega': ['gg', 'msd', 'ms', 'gen', 'scd', 'sat', 's32x', 'dc'],
    'atari': ['2600', '7800', '5200', 'aj', 'int'],
    'commodore': ['amig', 'c64', 'cd32'],
    'other': ['ouya', 'or', 'acpc', 'ast', 'apii', 'pce', 'zxs', 'lynx', 'ng', 'zxs', '3do', 'pcfx', 'ws', 'brw', 'cv', 'giz', 'msx', 'tg16', 'bbcm']
}

# Step 2: Flatten categories into a single list
all_items = []
for sublist in categories.values():
    for item in sublist:
        all_items.append(item)

# Step 3: Check for missing items
all_items_lower = [item.lower().strip() for item in all_items]
unique_values_lower = set(RawData['console'].str.lower().str.strip().unique())
missing_items = set(all_items_lower) - unique_values_lower

if missing_items:
    print(f"Missing items: {missing_items}")
else:
    print("All items are covered.")

# Step 4: Create conditions and values for np.select
conditions = []
for items in categories.values():
    conditions.append(RawData['console'].isin(items))

values = list(categories.keys())

# Step 5: Assign console manufacturers
RawData['console_mfg'] = np.select(conditions, values, default='unknown')

All items are covered.


In [7]:
RawData['console_mfg'].value_counts()

console_mfg
nintendo     16383
sony         14831
pc           12620
xbox          6461
sega          3078
other         1497
mobile        1224
atari          812
commodore       59
Name: count, dtype: int64

In [8]:
missing_consoles = RawData[RawData['console_mfg'] == "unknown"]['console']
print("Consoles with unknown manufacturers:")
missing_consoles.value_counts()

Consoles with unknown manufacturers:


Series([], Name: count, dtype: int64)

In [9]:
ConsoleToQuery = 'bbcm'
QueryResult = RawData[RawData['console'] == ConsoleToQuery]
print(f"Rows where console = '{ConsoleToQuery}':")
QueryResult.head(20)

Rows where console = 'bbcm':


,title,console,genre,publisher,developer,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,console_mfg
34229,Elite,bbcm,Misc,Frontier Developments Inc.,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,1984-09-20,other


In [10]:
GameToQuery = 'The Great Giana Sisters'
GameQueryResult = RawData[RawData['title'] == GameToQuery]
GameQueryResult.head(20)

,title,console,genre,publisher,developer,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,console_mfg
38407,The Great Giana Sisters,msx,Misc,Rainbow Arts,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,1987-01-01,other
38408,The Great Giana Sisters,ast,Misc,Rainbow Arts,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,1987-01-01,other
38409,The Great Giana Sisters,amig,Misc,Rainbow Arts,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,1987-01-01,commodore
38410,The Great Giana Sisters,c64,Misc,Rainbow Arts,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,1987-01-01,commodore
38411,The Great Giana Sisters,brw,Misc,Rainbow Arts,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,1987-01-01,other
41938,The Great Giana Sisters,acpc,Platform,Rainbow Arts,Time Warp Productions,0.0,0.0,0.0,0.0,0.0,0.0,1987-01-01,other


In [11]:
categoriesList = []
for manufacturer, consoles in categories.items():
    for console in consoles:
        categoriesList.append({'manufacturer': manufacturer, 'console': console})

# Converting the list to a DataFrame
mfg_list = pd.DataFrame(categoriesList)

# Grouping by 'manufacturer' and aggregating consoles into lists
grouped_series = mfg_list.groupby('manufacturer')['console'].apply(list)

# Converting the grouped Series to Markdown
markdown_table = MarkDownSeries(grouped_series)

print(markdown_table)

| manufacturer | console |
|---|---|
| atari | 2600, 7800, 5200, aj, int |
| commodore | amig, c64, cd32 |
| mobile | ios, and, winp, ngage, mob |
| nintendo | 3ds, dsiw, dsi, ds, wii, wiiu, ns, gb, gba, nes, snes, gbc, n64, vb, gc, vc, ww |
| other | ouya, or, acpc, ast, apii, pce, zxs, lynx, ng, zxs, 3do, pcfx, ws, brw, cv, giz, msx, tg16, bbcm |
| pc | linux, osx, pc, arc, all, fmt, c128, aco |
| sega | gg, msd, ms, gen, scd, sat, s32x, dc |
| sony | ps, ps2, ps3, ps4, ps5, psp, psv, psn, cdi |
| xbox | x360, xone, series, xbl, xb, xs |



| manufacturer | console |
|---|---|
| atari | 2600, 7800, 5200, aj, int |
| commodore | amig, c64, cd32 |
| mobile | ios, and, winp, ngage, mob |
| nintendo | 3ds, dsiw, dsi, ds, wii, wiiu, ns, gb, gba, nes, snes, gbc, n64, vb, gc, vc, ww |
| other | ouya, or, acpc, ast, apii, pce, zxs, lynx, ng, zxs, 3do, pcfx, ws, brw, cv, giz, msx, tg16, bbcm |
| pc | linux, osx, pc, arc, all, fmt, c128, aco |
| sega | gg, msd, ms, gen, scd, sat, s32x, dc |
| sony | ps, ps2, ps3, ps4, ps5, psp, psv, psn, cdi |
| xbox | x360, xone, series, xbl, xb, xs |
